In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [2]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [3]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test
using ITensors
ITensors.disable_warn_order()

14

In [4]:
seed = 124567890

Random.seed!(seed)

R = 12
sites = [Index(2, "n=$n") for n in 1:R]

Ψ = random_mps(sites; linkdims=20)
rindex = Set([rand(1:2, R) for _ in 1:20])

tensor = Array(reduce(*, Ψ), reverse(sites))
tensor = tensor ./ maximum(abs, tensor)

f(x) = tensor[x...]

localdims = fill(2, R)
initialpivots = MultiIndex[]
for _ in 1:100
    push!(initialpivots, [rand(1:d) for d in localdims])
end
#push!(initialpivots, [1, 1, 1, 1])
#push!(initialpivots, [2, 1, 1, 1])
#push!(initialpivots, [2, 1, 1, 1])
#push!(initialpivots, [2, 2, 1, 1])

tci = TensorCI2{Float64}(f, localdims, initialpivots)

TensorCI2{Float64}([[1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2], [2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1], [1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1], [2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1], [1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2], [2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1], [1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2], [1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1], [1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2]  …  [1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1], [1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1], [2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2], [2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2], [1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2], [1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2], [1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2], [2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1]], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.3847111302256151)

In [5]:
Iset, Jset = TCI.localpivots(tci, f; reltol=1e-8)
;

In [6]:
sitetensors = TCI.sitetensors(Float64, Iset, Jset, f, tci.localdims; orthocenter=length(tci.localdims) ÷ 2)
tt = TCI.TensorTrain(sitetensors)

LinearAlgebra.cond(P) = 40.61082562128508
LinearAlgebra.cond(P) = 61.11843570587926
LinearAlgebra.cond(P) = 66.62779435860001
LinearAlgebra.cond(P) = 10.461493486153996
LinearAlgebra.cond(P) = 2.9081103013858747
LinearAlgebra.cond(P) = 1.3963332840079217


(::TensorTrain{Float64, 3}) (generic function with 1 method)

In [7]:
Atensors = TCI.Atensors(Float64, Iset, Jset, f, tci.localdims)
Btensors = TCI.Btensors(Float64, Iset, Jset, f, tci.localdims)
;

LinearAlgebra.cond(P) = 1.5264099263372237
LinearAlgebra.cond(P) = 3.369412275756844
LinearAlgebra.cond(P) = 4.7822055225131725
LinearAlgebra.cond(P) = 12.00525994957196
LinearAlgebra.cond(P) = 35.72420824230781
LinearAlgebra.cond(P) = 40.61082562128508
LinearAlgebra.cond(P) = 61.11843570587926
LinearAlgebra.cond(P) = 66.62779435860001
LinearAlgebra.cond(P) = 10.461493486153996
LinearAlgebra.cond(P) = 2.9081103013858747
LinearAlgebra.cond(P) = 1.3963332840079217


In [8]:
function isleftnested(L, Iset, idx::Vector{Int})
    for b in 1:length(idx)
        if !(idx[1:b] ∈ Iset[b+1])
            return false
        end
    end
    return true
end

isleftnested (generic function with 1 method)

In [9]:
function isrightnested(L, Jset, idx::Vector{Int})
    for b in 1:length(idx)
        if !(idx[b:end] ∈ Jset[L + b - length(idx) - 1])
            return false
        end
    end
    return true
end

isrightnested (generic function with 1 method)

In [10]:
function rightenv(Jset::Vector{MultiIndex}, posinJset, Btensors)
    L = length(Btensors) + 1
    b = length(Jset[1])
    Bs_ = Btensors[L-1-b+1:L-1]
    only(reduce(*, [B_[:, σ, :] for (B_, σ) in zip(Bs_, Jset[posinJset])])[posinJset])
end

rightenv (generic function with 1 method)

In [11]:
rightenv(Jset[1], 1, Btensors)

1.0000000000000013

In [12]:
for b in 1:R-1
    for (j_, j) in enumerate(Jset[b])
        n_ = isrightnested(R, Jset, j)
        if n_
            continue
        end
        e = rightenv(Jset[b], j_, Btensors)
        @show b, j, e
        if abs(e - 1.0) > 1e-10
            @show b, j, e, n_
        end
    end
end

(b, j, e) = (1, [2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1], 0.9999999999999987)
(b, j, e) = (2, [1, 2, 1, 2, 2, 1, 1, 2, 2, 1], 1.0000000000000018)
(b, j, e) = (2, [1, 1, 2, 1, 1, 2, 1, 1, 1, 2], 1.0000000000000013)
(b, j, e) = (2, [2, 2, 2, 2, 1, 1, 2, 1, 1, 1], 1.0)
(b, j, e) = (3, [2, 1, 1, 2, 2, 1, 2, 1, 1], 0.9999999999999989)
(b, j, e) = (3, [2, 2, 2, 1, 1, 2, 1, 1, 1], 1.0000000000000004)
(b, j, e) = (3, [1, 2, 1, 2, 2, 2, 1, 1, 1], 1.0000000000000002)
(b, j, e) = (3, [1, 1, 1, 1, 1, 2, 2, 2, 1], 1.0000000000000002)
(b, j, e) = (4, [2, 2, 1, 1, 1, 2, 2, 1], 0.9999999999999993)
(b, j, e) = (4, [2, 2, 1, 1, 1, 1, 2, 2], 0.9999999999999996)
(b, j, e) = (4, [2, 1, 1, 2, 1, 2, 1, 2], 1.000000000000003)
(b, j, e) = (4, [1, 2, 1, 2, 2, 1, 1, 1], 1.0)
(b, j, e) = (4, [1, 1, 1, 1, 1, 2, 2, 2], 0.9999999999999996)
(b, j, e) = (4, [1, 1, 1, 1, 2, 2, 2, 1], 1.0000000000000029)
(b, j, e) = (5, [2, 1, 1, 1, 2, 2, 1], 0.9999999999999998)
(b, j, e) = (5, [2, 1, 2, 2, 1, 1, 2], 1.0000000000000013)
(b, j,

In [13]:
for b in 2:2
    for i in Iset[b+1], j in Jset[b]
        p = vcat(i, j)
        l_ = isleftnested(length(p), Iset, i)
        r_ = isrightnested(length(p), Jset, j)
        println("$i $j $(l_) $(r_)")
    end
end

[2, 2] [2, 1, 1, 1, 1, 1, 1, 2, 1, 1] true true
[2, 2] [1, 2, 1, 2, 2, 1, 1, 2, 2, 1] true false
[2, 2] [1, 1, 2, 1, 1, 2, 1, 1, 1, 2] true false
[2, 2] [2, 2, 2, 2, 1, 1, 2, 1, 1, 1] true false
[1, 2] [2, 1, 1, 1, 1, 1, 1, 2, 1, 1] true true
[1, 2] [1, 2, 1, 2, 2, 1, 1, 2, 2, 1] true false
[1, 2] [1, 1, 2, 1, 1, 2, 1, 1, 1, 2] true false
[1, 2] [2, 2, 2, 2, 1, 1, 2, 1, 1, 1] true false
[1, 1] [2, 1, 1, 1, 1, 1, 1, 2, 1, 1] true true
[1, 1] [1, 2, 1, 2, 2, 1, 1, 2, 2, 1] true false
[1, 1] [1, 1, 2, 1, 1, 2, 1, 1, 1, 2] true false
[1, 1] [2, 2, 2, 2, 1, 1, 2, 1, 1, 1] true false
[2, 1] [2, 1, 1, 1, 1, 1, 1, 2, 1, 1] true true
[2, 1] [1, 2, 1, 2, 2, 1, 1, 2, 2, 1] true false
[2, 1] [1, 1, 2, 1, 1, 2, 1, 1, 1, 2] true false
[2, 1] [2, 2, 2, 2, 1, 1, 2, 1, 1, 1] true false


In [14]:
# Interpolation error on pivot matrices
for b in 1:length(tci)-1
    for i in Iset[b+1], j in Jset[b]
        p = vcat(i, j)
        if abs(TCI.evaluate(tt, p) - f(p)) > 1e-10
            @show b, i, j
        end
    end
end

In [15]:
function checknesting(L, Iset, Jset, pivot)
    leftnested = fill(false, L-1)
    rightnested = fill(false, L-1)
    for b in 1:L-1
        if pivot[1:b] ∈ Iset[b+1]
            leftnested[b] = true
        else
            break
        end
    end
    for b in reverse(1:L-1)
        if pivot[b+1:end] ∈ Jset[b]
            rightnested[b] = true
        else
            break
        end
    end

    fullnestedbond = Int[]
    for b in 1:L-1
        if leftnested[b] && rightnested[b]
            push!(fullnestedbond, b)
        end
    end
    return fullnestedbond
end

checknesting (generic function with 1 method)

In [16]:
for b in 1:length(tci)-1
    for i in Iset[b+1], j in Jset[b]
        p = vcat(i, j)
        if abs(TCI.evaluate(tt, p) - f(p)) > 1e-10
            @show b, i, j
        end
        #res = checknesting(length(tci), Iset, Jset, p)
        #if isempty(res)
            #@show "not nested", b, i, j
            #@show abs(TCI.evaluate(tt, p) - f(p))
        #end
    end
end

In [17]:
function checknesting2(L, Iset, Jset, pivot)
    leftnested = fill(false, L-1)
    rightnested = fill(false, L-1)
    for b in 1:L-1
        if pivot[1:b] ∈ Iset[b+1]
            leftnested[b] = true
        end
    end
    for b in reverse(1:L-1)
        if pivot[b+1:end] ∈ Jset[b]
            rightnested[b] = true
        end
    end
    return leftnested, rightnested
end
#for i in initialpivots
    #@show i, checknesting2(R, Iset, Jset, i)
#end

checknesting2 (generic function with 1 method)

In [18]:
function checknesting(L, Iset, Jset, pivot)
    leftnested = fill(false, L-1)
    rightnested = fill(false, L-1)
    for b in 1:L-1
        if pivot[1:b] ∈ Iset[b+1]
            leftnested[b] = true
        else
            break
        end
    end
    for b in reverse(1:L-1)
        if pivot[b+1:end] ∈ Jset[b]
            rightnested[b] = true
        else
            break
        end
    end

    fullnestedbond = Int[]
    for b in 1:L-1
        if leftnested[b] && rightnested[b]
            push!(fullnestedbond, b)
        end
    end
    return fullnestedbond
end

checknesting (generic function with 1 method)

In [19]:
tt = TCI.TensorTrain(TCI.sitetensors(tci, f))

for b in 1:length(tci)-1
    for i in Iset[b+1], j in Jset[b]
        p = vcat(i, j)
        if abs(TCI.evaluate(tt, p) - f(p)) > 1e-10
            @show b, i, j
        end
        #res = checknesting(length(tci), Iset, Jset, p)
        #if isempty(res)
            #@show "not nested", b, i, j
            #@show abs(TCI.evaluate(tt, p) - f(p))
        #end
    end
end

orthocenter = 6


In [20]:
for p in initialpivots
    bs = checknesting(R, Iset, Jset, p)
    println("$p $bs")
    if isempty(bs)
        @error "$p is not full nested at any bond!"
    end
end

[1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2] Int64[]


┌ Error: [1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:5
┌ Error: [1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:5
┌ Error: [1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:5
┌ Error: [1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:5
┌ Error: [1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_c

[2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1] [4]
[1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1] Int64[]
[2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1] [5]
[1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2] Int64[]
[2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1] [3, 4, 5, 6]
[2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1] [3, 4, 5]
[1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2] Int64[]
[1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1] Int64[]
[1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2] Int64[]
[1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1] Int64[]
[1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2] Int64[]
[1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1] Int64[]
[2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1] Int64[]
[1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1] Int64[]
[1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2] Int64[]
[1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2] Int64[]
[1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2] Int64[]
[1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1] Int64[]
[1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2] [7]
[2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1] [4, 5]
[1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1] Int64[]
[1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1] [4]
[1, 2, 1, 2, 2, 1, 1

┌ Error: [1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:5
┌ Error: [2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:5
┌ Error: [1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:5
┌ Error: [1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:5
┌ Error: [1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_c

In [21]:
for b in 1:length(tci)-1
    @show b
    @show Iset[b+1]
    @show Jset[b]
    println("")
end

b = 1
Iset[b + 1] = [[2], [1]]
Jset[b] = [[2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1], [2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1]]

b = 2
Iset[b + 1] = [[2, 2], [1, 2], [1, 1], [2, 1]]
Jset[b] = [[2, 1, 1, 1, 1, 1, 1, 2, 1, 1], [1, 2, 1, 2, 2, 1, 1, 2, 2, 1], [1, 1, 2, 1, 1, 2, 1, 1, 1, 2], [2, 2, 2, 2, 1, 1, 2, 1, 1, 1]]

b = 3
Iset[b + 1] = [[1, 1, 1], [1, 2, 1], [2, 2, 2], [1, 1, 2], [1, 2, 2], [2, 1, 2], [2, 1, 1], [2, 2, 1]]
Jset[b] = [[1, 2, 1, 1, 1, 1, 2, 1, 1], [1, 1, 1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 2, 2, 1, 2, 1, 1], [2, 2, 2, 2, 2, 2, 2, 1, 1], [1, 2, 2, 2, 1, 2, 1, 1, 1], [2, 2, 2, 1, 1, 2, 1, 1, 1], [1, 2, 1, 2, 2, 2, 1, 1, 1], [1, 1, 1, 1, 1, 2, 2, 2, 1]]

b = 4
Iset[b + 1] = [[1, 1, 1, 1], [1, 1, 2, 2], [2, 2, 1, 1], [1, 2, 1, 1], [2, 1, 2, 1], [2, 2, 2, 2], [2, 2, 2, 1], [2, 1, 2, 2], [1, 2, 2, 2], [2, 1, 1, 1], [1, 1, 1, 2], [2, 1, 1, 2], [1, 1, 2, 1], [1, 2, 1, 2], [2, 2, 1, 2], [1, 2, 2, 1]]
Jset[b] = [[2, 1, 1, 1, 1, 2, 1, 1], [2, 2, 1, 1, 1, 2, 2, 1], [2, 2, 2, 2, 2, 2, 1, 1], [1, 1, 

In [22]:
#pivot = [2, 2, 1, 1]
pivot = [1, 1, 1, 1]
sitetensors_p = [t[:, p, :] for (p, t) in zip(pivot, tt)]
for t in sitetensors_p
    @show size(t)
end

size(t) = (1, 2)
size(t) = (2, 4)
size(t) = (4, 8)
size(t) = (8, 16)
